In [15]:
tasks_path = "data/test_data/tasks.json"
splits_path = "data/splits/splits_test.json"

import json

from src import config

d_test_task = json.load(open(tasks_path, "r"))
print(d_test_task["monolingual"]["ara"].keys())

for lang in config.LANGS + ["pol", "tur"]:
    print(lang)
    print(d_test_task["monolingual"][lang].keys())
    d_test_task["monolingual"][lang]["posts_train"] = []
    d_test_task["monolingual"][lang]["posts_dev"] = d_test_task["monolingual"][lang]["posts_test"]

d_test_task["crosslingual"]["posts_train"] = []
d_test_task["crosslingual"]["posts_dev"] = d_test_task["crosslingual"]["posts_test"]

d_test_task["crosslingual"].keys()

json.dump(d_test_task, open(splits_path, "w"))

dict_keys(['fact_checks', 'posts_test'])
eng
dict_keys(['fact_checks', 'posts_test'])
fra
dict_keys(['fact_checks', 'posts_test'])
deu
dict_keys(['fact_checks', 'posts_test'])
por
dict_keys(['fact_checks', 'posts_test'])
spa
dict_keys(['fact_checks', 'posts_test'])
tha
dict_keys(['fact_checks', 'posts_test'])
msa
dict_keys(['fact_checks', 'posts_test'])
ara
dict_keys(['fact_checks', 'posts_test'])
pol
dict_keys(['fact_checks', 'posts_test'])
tur
dict_keys(['fact_checks', 'posts_test'])


In [39]:
len(d_test_task["monolingual"]["tur"]["posts_dev"])

500

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
nlp = spacy.load("en_core_web_lg")

from tqdm import tqdm
tqdm.pandas()   

from src import config
from src.datasets import TextConcatFactCheck, TextConcatPosts
from src.utils import cleaning_spacy, cleaning_spacy_batch

tasks_path = "data/splits/splits_test.json"
posts_path = "data/test_data/posts.csv"
fact_checks_path = "data/test_data/fact_checks.csv"
gs_path = config.GS_PATH
# lang = 'tha'
task_name = "monolingual"

print("Loading Fact Checks...")
fc = TextConcatFactCheck(fact_checks_path, tasks_path=tasks_path, task_name=task_name, lang=lang, version="english")
print("Loading Fact Checks (English + Clean)...")
# fc_eng = TextConcatFactCheck(fact_checks_path, tasks_path=tasks_path, task_name=task_name, lang=lang, version="english", cleaning_function=lambda x: cleaning_spacy_batch(x, nlp))

print("Loading Posts...")
posts = TextConcatPosts(posts_path, tasks_path=tasks_path, task_name=task_name, lang=lang, gs_path=gs_path, version="english")
# print("Loading Posts (English + Clean)...")
# posts_eng = TextConcatPosts(posts_path, tasks_path=tasks_path, task_name=task_name, lang=lang, gs_path=gs_path, version="english", cleaning_function=lambda x: cleaning_spacy_batch(x, nlp))

from sentence_transformers import SentenceTransformer, CrossEncoder
from src.models import EmbeddingModel
from time import time
import torch

df_posts_dev = posts.df_dev
n_candidates = 100
output_k = 10

teacher_model_name = '/gpfs/projects/bsc14/abecerr1/hub/models--Snowflake--snowflake-arctic-embed-l-v2.0/snapshots/edc2df7b6c25794b340229ca082e7c78782e6374'
teacher_model_base = SentenceTransformer(teacher_model_name, device="cuda", trust_remote_code=True)
emb_batch_size = 256
teacher_model = EmbeddingModel(teacher_model_base, df_fc, batch_size=emb_batch_size)

reranker_model_name = 'official/contrastive/snowflake_mv2/models/monolingual_tha_reranker'
rerank_model = CrossEncoder(reranker_model_name, num_labels=1, max_length=1024, trust_remote_code=True)

print("Predicting...")
time_start = time()
with torch.no_grad():
    arr_cands = teacher_model.predict(df_posts_dev["full_text"].values)[:,:n_candidates]

# df_dev_out = df_posts_dev[["full_text", "gs"]].copy()
ls_preds = []
for i in tqdm(range(len(df_posts_dev))):
    ranked_vals = rerank_model.rank(df_posts_dev["full_text"].values[i], df_fc.loc[arr_cands[i], "full_text"].values, show_progress_bar=False)
    ls_preds.append([int(arr_cands[i, dd["corpus_id"]]) for dd in ranked_vals][:output_k])

df_posts_dev["preds"] = ls_preds
# df_posts_dev["preds"] = model.predict(df_posts_dev["full_text"].values).tolist()
print(f"Time taken: {time() - time_start:.2f}s\n")

Loading Fact Checks...
Loading Fact Checks (English + Clean)...
Loading Posts...


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
from sentence_transformers import SentenceTransformer, CrossEncoder
from tqdm import tqdm
from time import time
import torch
from src import config
from src.datasets import TextConcatFactCheck, TextConcatPosts
from src.utils import cleaning_spacy, cleaning_spacy_batch
from src.models import EmbeddingModel

def predict_fact_checks(reranker_model_name, task_name="monolingual", n_candidates=100, output_k=10, lang="eng"):
    """
    Function to predict the top fact-check candidates for posts.

    Args:
    - model_path (str): Path to the pre-trained model.
    - task_name (str): Task name, default is "monolingual".
    - n_candidates (int): Number of candidate fact-checks to retrieve.
    - output_k (int): Number of top-ranked fact-checks to output.
    - lang (str): Language code for the task.

    Returns:
    - pd.DataFrame: DataFrame containing posts with predictions.
    """
    
    # Hardcoded paths
    fact_checks_path = "data/test_data/fact_checks.csv"
    posts_path = "data/test_data/posts.csv"
    tasks_path = "data/splits/splits_test.json"
    gs_path = config.GS_PATH

    # Load Fact Checks
    print("Loading Fact Checks...")
    fc = TextConcatFactCheck(fact_checks_path, tasks_path=tasks_path, task_name=task_name, lang=lang, version="english")

    # Load Posts
    print("Loading Posts...")
    posts = TextConcatPosts(posts_path, tasks_path=tasks_path, task_name=task_name, lang=lang, gs_path=gs_path, version="english")

    # Initialize Models
    print("Initializing Models...")
    teacher_model_name = '/gpfs/projects/bsc14/abecerr1/hub/models--Snowflake--snowflake-arctic-embed-l-v2.0/snapshots/edc2df7b6c25794b340229ca082e7c78782e6374'
    teacher_model_base = SentenceTransformer(teacher_model_name, device="cuda", trust_remote_code=True)
    emb_batch_size = 256
    teacher_model = EmbeddingModel(teacher_model_base, fc.df, batch_size=emb_batch_size)

    rerank_model = CrossEncoder(reranker_model_name, num_labels=1, max_length=1024, trust_remote_code=True)

    # Predict Fact-Check Candidates
    print("Predicting...")
    time_start = time()
    with torch.no_grad():
        arr_cands = teacher_model.predict(posts.df_dev["full_text"].values)[:,:n_candidates]

    # Initialize List for Predictions
    ls_preds = []
    for i in tqdm(range(len(posts.df_dev))):
        ranked_vals = rerank_model.rank(posts.df_dev["full_text"].values[i], fc.df.loc[arr_cands[i], "full_text"].values, show_progress_bar=False)
        ls_preds.append([int(arr_cands[i, dd["corpus_id"]]) for dd in ranked_vals][:output_k])

    # Add Predictions to DataFrame
    posts.df_dev["preds"] = ls_preds
    print(f"Time taken: {time() - time_start:.2f}s\n")

    d_out = posts.df_dev["preds"].to_dict()

    return d_out

# Example usage:
reranker_model_name = 'official/contrastive/snowflake_mv2/models/monolingual_tha_reranker'
predictions = predict_fact_checks(reranker_model_name, task_name="monolingual", n_candidates=100, output_k=10, lang="tha")
print(predictions)


Loading Fact Checks...
Loading Posts...
Initializing Models...


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

/gpfs/projects/bsc14/scratch/.conda/factcheck/lib/python3.10/site-packages/flash_attn/ops/triton/layer_norm.py:985: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(
/gpfs/projects/bsc14/scratch/.conda/factcheck/lib/python3.10/site-packages/flash_attn/ops/triton/layer_norm.py:1044: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, dout, *args):


Predicting...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 183/183 [00:09<00:00, 20.17it/s]


Time taken: 24.22s

{28399: [193416, 193411, 345386, 193421, 352500, 193415, 193395, 350366, 286514, 193393], 28649: [258207, 273044, 193368, 193477, 355538, 148640, 193244, 193434, 193489, 193464], 28688: [193345, 263035, 193314, 193257, 193325, 258388, 365049, 366079, 113776, 193351], 28724: [193546, 193355, 214005, 193548, 193542, 267395, 33945, 231885, 193370, 193547], 28964: [193333, 193578, 193435, 300143, 352500, 193471, 193382, 193415, 193341, 280080], 29092: [193559, 193351, 193557, 193558, 148640, 148641, 193582, 193463, 193380, 193475], 29093: [193471, 193388, 193274, 193296, 193262, 193313, 193300, 193315, 193585, 148640], 29441: [193540, 193342, 337647, 339971, 231885, 193603, 339958, 193431, 312495, 350107], 29783: [193357, 8145, 350366, 193485, 270176, 193437, 193322, 193406, 193497, 193449], 29941: [193536, 193400, 193427, 357213, 193416, 314903, 193541, 348832, 193562, 193529], 30048: [243918, 304341, 193397, 193326, 193583, 193310, 193361, 193391, 364379, 193548], 302

In [2]:
df_fc = fc.df

In [ ]:


# d_out.update(df_posts_ded_outv["preds"].to_dict())

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

/gpfs/projects/bsc14/scratch/.conda/factcheck/lib/python3.10/site-packages/flash_attn/ops/triton/layer_norm.py:985: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(
/gpfs/projects/bsc14/scratch/.conda/factcheck/lib/python3.10/site-packages/flash_attn/ops/triton/layer_norm.py:1044: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, dout, *args):


Predicting...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 183/183 [00:09<00:00, 19.59it/s]

Time taken: 24.54s



In [4]:
df_posts_dev

,ocr,verdicts,text,lan,fb,tw,ig,full_text,gs,preds
post_id,,,,,,,,,,
28399,,False information.,"Try it, right? This image was created by Prof....",tha,1,0,0,"Try it, right? This image was created by Prof...",[],"[193416, 193411, 345386, 193421, 352500, 19341..."
28649,THAI MOVE INSTITUTE The Finnish government urg...,Missing context,The increasingly tense situation has escalated...,tha,1,0,0,THAI MOVE INSTITUTE The Finnish government urg...,[],"[258207, 273044, 193368, 193477, 355538, 14864..."
28688,FGT WITS Canuck. FaceApp FaceApp,Partly false,Hot pineapple juice..............................,tha,1,0,0,FGT WITS Canuck. FaceApp FaceApp Hot pineapple...,[],"[193345, 263035, 193314, 193257, 193325, 25838..."
28724,,Partly false,I&#39;m sure Singaporeans are smarter than the...,tha,1,0,0,I&#39;m sure Singaporeans are smarter than th...,[],"[193546, 193355, 214005, 193548, 193542, 26739..."
28964,,Partly false information.,Wasan! Stop online learning but have students ...,tha,1,0,0,Wasan! Stop online learning but have students...,[],"[193333, 193578, 193435, 300143, 352500, 19347..."
...,...,...,...,...,...,...,...,...,...,...
78627,Today Yesterday TikTok Ibraheem 1997,,"A famous warrior on social media, creating an ...",tha,0,1,0,Today Yesterday TikTok Ibraheem 1997 A famous ...,[],"[332789, 331622, 339963, 337842, 337647, 33997..."
78630,Abutofal,,"Behind the scenes of the Palestinian filming, ...",tha,0,1,0,Abutofal Behind the scenes of the Palestinian ...,[],"[339963, 344563, 314903, 337842, 312495, 33162..."
78753,,,Many pets are facing a fate as floodwaters bre...,tha,0,1,0,Many pets are facing a fate as floodwaters br...,[],"[283561, 193375, 193492, 214494, 214383, 19328..."


In [36]:
import json
d_target = json.load(open("data/test_data/monolingual_predictions.json", "r"))
d_pred = json.load(open("official/contrastive/snowflake_mv2/predictions/monolingual_predictions.json", "r"))

assert d_target.keys() == d_pred.keys()

AssertionError: 

In [37]:
import numpy as np
np.intersect1d(list(d_target.keys()), list(d_pred.keys())).shape

(1000,)

In [34]:
len(d_pred.keys())

4000

In [35]:
len(d_target.keys())

4000

In [17]:
sorted(d_target.keys())

['28094',
 '28095',
 '28102',
 '28112',
 '28150',
 '28158',
 '28173',
 '28178',
 '28193',
 '28204',
 '28205',
 '28219',
 '28221',
 '28222',
 '28227',
 '28253',
 '28258',
 '28263',
 '28264',
 '28274',
 '28276',
 '28277',
 '28319',
 '28329',
 '28334',
 '28342',
 '28361',
 '28370',
 '28387',
 '28393',
 '28399',
 '28401',
 '28434',
 '28451',
 '28456',
 '28468',
 '28502',
 '28513',
 '28543',
 '28546',
 '28553',
 '28567',
 '28597',
 '28602',
 '28637',
 '28641',
 '28649',
 '28650',
 '28668',
 '28671',
 '28687',
 '28688',
 '28697',
 '28699',
 '28701',
 '28710',
 '28716',
 '28724',
 '28734',
 '28749',
 '28751',
 '28793',
 '28803',
 '28856',
 '28864',
 '28872',
 '28878',
 '28881',
 '28892',
 '28894',
 '28898',
 '28936',
 '28941',
 '28950',
 '28964',
 '28969',
 '28970',
 '28989',
 '28998',
 '29000',
 '29012',
 '29013',
 '29017',
 '29032',
 '29037',
 '29046',
 '29051',
 '29067',
 '29068',
 '29083',
 '29085',
 '29090',
 '29091',
 '29092',
 '29093',
 '29094',
 '29097',
 '29111',
 '29124',
 '29140',


{'29970': [60639,
  60897,
  60955,
  145923,
  274921,
  60328,
  146048,
  60327,
  60954,
  60640],
 '30010': [64217,
  64346,
  60697,
  91339,
  311855,
  60888,
  60953,
  60899,
  60755,
  283794],
 '30157': [293909,
  60863,
  57014,
  275197,
  288166,
  275051,
  26278,
  214354,
  86604,
  114247],
 '30851': [156787,
  275445,
  146003,
  145984,
  282720,
  274921,
  60709,
  234311,
  249260,
  60837],
 '31031': [275344,
  140565,
  145827,
  60687,
  79142,
  275208,
  287437,
  145869,
  60686,
  309179],
 '31278': [126002,
  82366,
  64331,
  145923,
  276990,
  275317,
  274921,
  258734,
  12375,
  157334],
 '31630': [64337,
  64341,
  372388,
  372348,
  64403,
  274946,
  345981,
  31704,
  60639,
  60971],
 '32438': [46166,
  307204,
  77196,
  146011,
  257803,
  46167,
  40967,
  372261,
  27611,
  347761],
 '32740': [60614,
  301167,
  280364,
  364736,
  157457,
  60613,
  60612,
  360977,
  60868,
  277601],
 '34612': [145762,
  145941,
  365225,
  146023,
  3

In [18]:
sorted(d_pred.keys())

['28094',
 '28095',
 '28102',
 '28178',
 '28193',
 '28204',
 '28219',
 '28263',
 '28264',
 '28513',
 '28546',
 '28710',
 '28749',
 '28894',
 '29032',
 '29037',
 '29091',
 '29094',
 '29097',
 '29161',
 '29194',
 '29215',
 '29343',
 '29366',
 '29657',
 '29677',
 '29685',
 '29696',
 '29805',
 '29942',
 '29964',
 '29970',
 '30000',
 '30010',
 '30157',
 '30223',
 '30318',
 '30413',
 '30437',
 '30497',
 '30502',
 '30631',
 '30646',
 '30658',
 '30821',
 '30851',
 '31031',
 '31081',
 '31088',
 '31278',
 '31587',
 '31614',
 '31630',
 '31692',
 '31726',
 '31786',
 '32014',
 '32422',
 '32426',
 '32438',
 '32487',
 '32685',
 '32740',
 '32749',
 '32796',
 '32974',
 '33112',
 '33132',
 '33220',
 '33231',
 '33250',
 '33256',
 '33291',
 '33380',
 '33407',
 '33430',
 '33461',
 '33547',
 '33714',
 '33735',
 '33764',
 '33842',
 '34036',
 '34041',
 '34136',
 '34196',
 '34268',
 '34391',
 '34562',
 '34612',
 '34654',
 '34708',
 '34775',
 '34804',
 '34817',
 '34821',
 '34825',
 '34835',
 '34848',
 '34980',
